Con este código podremos llevar a cabo el web Scrapping para obtener los trabajos de data science de Bumerang

Pasos antes de empezar: 
1. Generar un nuevo environment para evitar problemas de versiones entre las librerias que usaremos
2. La versión de python que utilizarán será la 3.10
2. Instalar las librerias con sus respectivas versiones que encontrará en el archivo adjunto "requirements.txt." la carpeta
3. Asegurarse de que el environment este activado


# 1. Instalar las librerias en anaconda prompt o con !pip. 

# 2. Calling libraries

In [ ]:
# Manipular la página web 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException , StaleElementReferenceException, WebDriverException, ElementClickInterceptedException


# it allows you to work with differen versions of drivers
# We call ChromeDriver
# from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
import csv


# 3. Inicializar la web 

In [4]:
driver = webdriver.Chrome()
# driver = webdriver.Firefox()
url = "https://www.bumeran.com.pe/empleos.html"
driver.get(url)
driver.maximize_window()

In [5]:
# Esperar hasta que el botón "Fecha de publicación" esté presente y hacer clic
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Fecha de publicación')]"))).click()

# Esperar hasta que el botón "Menor a 15 días" esté presente y hacer clic
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Menor a 15')]"))).click()



In [6]:
# Esperar hasta que el botón "Área" esté presente y hacer clic
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Área')]"))).click()
# Esperar hasta que aparezca el campo de búsqueda por NAME
campo_busqueda = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, "search-facets"))
)

# Escribir "Tecnología, Sistemas y telecomunicaciones" en la caja de búsqueda
campo_busqueda.send_keys("Tecnología, Sistemas y telecomunicaciones")

boton_tecnologia = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Tecnología, Sistemas y Telecomunicaciones')]"))
)
boton_tecnologia.click()

In [7]:
# Esperar y hacer clic en el botón "Subárea"
boton_subarea = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Sub')]"))
).click()

# Esperar hasta que el botón "Programación" esté visible y clickeable
boton_programacion = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Programación')]"))
).click()

In [8]:
# Hacer clic en el botón "Departamento" para abrir el menú desplegable
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Departamento')]"))
).click()

# Esperar hasta que la caja de búsqueda esté visible
campo_busqueda = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.NAME, "search-facets"))
)

# Hacer clic en la caja de búsqueda para activarla
campo_busqueda.click()

# Escribir "Lima" en la caja de búsqueda
campo_busqueda.send_keys("Lima")

# Esperar a que aparezca el botón con "Lima" y hacer clic en él
boton_lima = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Lima')]"))
)
boton_lima.click()


In [9]:
# Esperar a que el botón "Carga horaria" sea clickeable y hacer clic
boton_carga_horaria = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Carga horaria')]"))
)
boton_carga_horaria.click()

# Esperar a que el botón "Full-time" sea clickeable y hacer clic
boton_full_time = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Full-time')]"))
)
boton_full_time.click()

time.sleep(2)

## FASE 1: RECOLECTAR LOS LINKS

In [10]:
job_links = set()  # Conjunto para almacenar enlaces únicos de empleos

while True:
    try:
        # Esperar que los enlaces de empleo carguen
        WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
        links = driver.find_elements(By.TAG_NAME, "a")
        
        # Extraer los enlaces que contienen "/empleos/"
        page_job_links = {link.get_attribute("href") for link in links if link.get_attribute("href") and "/empleos/" in link.get_attribute("href")}
        job_links.update(page_job_links)  # Agregar los nuevos enlaces a la lista total
        
        print(f"🔗 Anuncios en esta página ({len(page_job_links)} encontrados):")
        for link in page_job_links:
            print(link)

        # Intentar hacer clic en el botón siguiente
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "icon-icon-light-caret-right"))
            )
            next_button.click()
            print("➡️ Se hizo clic en el botón siguiente.")
            
            # Pausa para que la página cargue completamente antes de continuar
            time.sleep(2)  
            
            # Esperar a que la nueva página se cargue antes de buscar los anuncios
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))

        except (TimeoutException, NoSuchElementException, ElementClickInterceptedException):
            print("❌ No hay más páginas disponibles.")
            break  # Salir del bucle si no se puede hacer más clics en "siguiente"

    except Exception as e:
        print(f"⚠️ Error inesperado: {e}")
        break

🔗 Anuncios en esta página (20 encontrados):
https://www.bumeran.com.pe/empleos/frontend-developer-react-maseka-consulting-s.a.c.-1116739942.html
https://www.bumeran.com.pe/empleos/analista-programador-indra-peru-1116752132.html
https://www.bumeran.com.pe/empleos/desarrollador-front-ios-summit-s.a.c-1116742593.html
https://www.bumeran.com.pe/empleos/desarrollador-en-innovacion-y-automatizacion-csti-corp-1116751489.html
https://www.bumeran.com.pe/empleos/frontend-developer-react-manpowergroup-peru-1116752520.html
https://www.bumeran.com.pe/empleos/analista-programador-de-sistemas-tecsur.s.a-1116752494.html
https://www.bumeran.com.pe/empleos/analista-programador-power-builder-junior-1116745817.html
https://www.bumeran.com.pe/empleos/backend-developer-nodejs-manpowergroup-peru-1116752515.html
https://www.bumeran.com.pe/empleos/programador-frontend-junior-presencial-manpowergroup-peru-1116752262.html
https://www.bumeran.com.pe/empleos/jefe-de-sistemas-ti-experis-peru-1116752439.html
https:/

## FASE 2: Obtener los datos de cada página

In [ ]:


# Lista para almacenar información de cada oferta
job_data = []

def get_district(driver):
    """Extrae el distrito desde el SEGUNDO <h2> dentro de la ficha del empleo."""
    district = "No disponible"
    try:
        # Esperar a que haya al menos dos elementos h2 en la página
        h2_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#ficha-detalle h2"))
        )

        # Verificar que haya al menos dos elementos y obtener el segundo
        if len(h2_elements) > 1:
            district = h2_elements[1].text.strip().title()  # Capitaliza cada palabra
    except TimeoutException:
        print("Distrito no encontrado a tiempo.")
    except StaleElementReferenceException:
        print("Elemento de distrito se volvió obsoleto, intentando de nuevo...")

    return district

# Reintentar la extracción en caso de error
MAX_RETRIES = 3  # Intentos máximos por empleo

# Entrar a cada oferta de empleo
for job_link in job_links:
    attempts = 0
    while attempts < MAX_RETRIES:
        try:
            driver.get(job_link)
            print(f"Extrayendo datos de: {job_link} (Intento {attempts+1}/{MAX_RETRIES})")

            # Extraer título con WebDriverWait
            try:
                title_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                title = title_element.text.strip()
            except TimeoutException:
                title = "No disponible"

            # Extraer distrito
            district = get_district(driver)

            # Extraer descripción con WebDriverWait
            try:
                description_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="ficha-detalle"]/div[2]/div/div[1]/p'))
                )
                description = description_element.text.strip()
            except TimeoutException:
                description = "No disponible"

            # Extraer modalidad desde el enlace con WebDriverWait
            modality = "No disponible"
            try:
                modality_elements = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/empleos-modalidad-']"))
                )
                for mod in modality_elements:
                    href = mod.get_attribute("href")
                    match = re.search(r"-([^-]+)\.html", href)
                    if match:
                        modality = match.group(1).capitalize()
                        break
            except TimeoutException:
                pass

            # Guardar datos en la lista y salir del bucle de reintento
            job_data.append([title, district, description, modality, job_link])
            break  # Si todo funciona, sale del bucle de intentos

        except (TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException) as e:
            print(f"⚠️ Error en {job_link}: {e}")
            attempts += 1
            if attempts < MAX_RETRIES:
                print(f"🔄 Reintentando ({attempts}/{MAX_RETRIES})...")
                time.sleep(2)  # Espera antes de reintentar
            else:
                print(f"❌ Falló la extracción después de {MAX_RETRIES} intentos. Omitiendo este empleo.")

# Guardar los datos en un archivo CSV
csv_filename = "Websrapping_bumerang_results.csv"
with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Job Title", "Description ", "District ", "Work Mode","URL"])
    writer.writerows([[row[0], row[2], row[1], row[3], row[4]] for row in job_data])

print(f"✅ Se guardaron {len(job_data)} ofertas de empleo en {csv_filename}")




# Imprimir el total de anuncios encontrados en las páginas
print(f"\n✅ Total de anuncios de empleo encontrados: {len(job_links)}")

Extrayendo datos de: https://www.bumeran.com.pe/empleos/analista-programador-indra-peru-1116752132.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/desarrollador-.net-net-core-remoto-stefanini-peru-s.a-1116734987.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/frontend-developer-react-manpowergroup-peru-1116752520.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/desarrollador-fullstack-1116737978.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/analista-programador-de-sistemas-tecsur.s.a-1116752494.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/analista-programador-power-builder-junior-1116745817.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/analista-programador-front-end-presencial-canvia-1116751447.html (Intento 1/3)
Extrayendo datos de: https://www.bumeran.com.pe/empleos/backend-developer-nodejs-manpowergroup-peru-111675251

✅ Se guardaron 114 ofertas de empleo en Websrapping_bumerang_results.csv
